In [1]:
#!/usr/bin/env python3
"""
Patient Timeline Creator
Creates a comprehensive timeline for patients by combining data from multiple tables.
"""

import pandas as pd
import duckdb as db
import os
from datetime import datetime
from typing import Dict, List, Tuple, Optional

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:

def get_database_connection():
    """Create and return a connection to the clinisys_all database"""
    path_to_db = '../../database/huntington_data_lake.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def extract_timeline_data_for_patient(conn, test_prontuario):
    """Extract timeline data for a specific patient"""
    query = f"""
        SELECT * FROM gold.all_patients_timeline
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def get_finops_data(conn, test_prontuario):
    """Get finops data for a specific patient"""
    query = f"""
        SELECT * FROM gold.finops_summary
        WHERE prontuario = {test_prontuario}
    """
    return conn.execute(query).fetchdf()

def main(test_prontuario):
    """Main function to create patient timeline"""
    
    # Test with a single patient first
    # test_prontuario = 175583
    
    print(f"Creating timeline for patient: {test_prontuario}")
    
    # Get database connection
    conn = get_database_connection()
    
    try:
        # Extract timeline data
        timeline_df = extract_timeline_data_for_patient(conn, test_prontuario)
       
        finops_df = get_finops_data(conn, test_prontuario)
        return timeline_df, finops_df
        
    finally:
        conn.close()

In [3]:
if __name__ == "__main__":
    patient_id = 876_950
    patient_id = 825_890 # Claudia
    patient_id = 175_583 # Renata
    # patient_id = 220_783 # caso cabelud0:: - 26 tentativas
    # patient_id = 182_925 # caso completo
    patient_id = 162_173
    patient_id = 641_950 # 11 cycles without payment

    timeline_df, finops_df = main(patient_id)


Creating timeline for patient: 641950
Connected to database: ../../database/huntington_data_lake.duckdb
Database file exists: True


In [4]:
timeline_df

,prontuario,event_id,event_date,reference,reference_value,tentativa,unidade,resultado_tratamento,flag_date_estimated,additional_info
0,641950.0,1444655,2025-07-29,extrato_atendimentos,Retorno Pós Tratamento,None,<NA>,None,False,{}
1,641950.0,1445766,2025-07-16,extrato_atendimentos,Retorno Psicologia,None,<NA>,None,False,{}
2,641950.0,1444417,2025-07-16,extrato_atendimentos,Acompanhamento Beta hCG ***,None,<NA>,None,False,{}
3,641950.0,1444416,2025-07-14,extrato_atendimentos,Acompanhamento Beta hCG ***,None,<NA>,None,False,{}
4,641950.0,1438264,2025-07-04,extrato_atendimentos,Retorno Psicologia,None,<NA>,None,False,{}
5,641950.0,35391,2025-07-03,tratamentos,Ciclo de Congelados,11,7,Negativo,False,{}
6,641950.0,18126,2025-07-03,descongelamentos_embrioes,2835/24,None,<NA>,None,False,"{""CodDescongelamento"": ""1366/25"", ""Unidade"": ""7""}"
7,641950.0,1433904,2025-07-03,extrato_atendimentos,FET: Excedente Próprio,None,<NA>,None,False,{}
8,641950.0,1433890,2025-06-27,extrato_atendimentos,Retorno Pós ultrassom,None,<NA>,None,False,{}
9,641950.0,1426259,2025-06-27,extrato_atendimentos,US - Preparo endometrial,None,<NA>,None,False,{}


In [5]:
finops_df

,prontuario,cycle_with_transfer,cycle_without_transfer,cycle_total,fiv_initial_paid_count,fiv_initial_paid_total,fiv_extra_paid_count,fiv_extra_paid_total,fiv_paid_count,fiv_paid_total,timeline_first_date,timeline_last_date,billing_first_date,billing_last_date,flag_cycles_no_payments,flag_more_cycles_than_payments,flag_no_cycles_but_payments,flag_less_cycles_than_payments
0,641950.0,7,4,11,20,81496.0,0,0.0,20,81496.0,2023-05-09,2025-07-03,2023-05-12,2024-09-10,0,0,0,1


In [6]:
timeline_df.loc[5, 'additional_info']

'{}'